In [1]:
import pandas as pd
import requests
from tqdm import tqdm
from time import sleep
from Bio import Entrez, SeqIO
import re

input_csv = r"C:\Users\poker\Downloads\big-search_no-filters-40%similarity_70-80%nodes\OvoA_10k_Blast_search_noFilter_80AST_80%_SSN_topmiddleClusterIDs.csv"
output_csv = "protein_synteny_topmiddleClusterIDs_output.csv"

Entrez.email = "phitro@bu.edu" # Set email for NCBI Entrez, for responsible use needed
search_genes = ['selA', 'selB', 'selC', 'selD', 'SenA', 'SenB', 'SenC', 'SenD', 'EgtA', 'EgtB', 'EgtC', 'EgtD', 'EgtE', 'OvoA']

CHECKPOINT_EVERY = 20  # How often to write intermediate result tp CSVs


# Helper functions

In [28]:
def get_uniprot_mappings(uniprot_id):
    """Return RefSeq protein, RefSeq nucleotide, EMBL nucleotide, EMBL protein for a UniProt ID."""
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    r = requests.get(url, timeout=10)
    if r.status_code != 200:
        return None, None, None, None
    data = r.json()
    refseq_prot, refseq_nt, embl_nt, embl_prot = None, None, None, None
    for dbref in data.get('uniProtKBCrossReferences', []):
        if dbref['database'] == "RefSeq":
            refseq_prot = dbref.get('id') or refseq_prot
            for prop in dbref.get('properties', []):
                if prop['key'] == 'NucleotideSequenceId':
                    refseq_nt = prop['value']
        if dbref['database'] == "EMBL":
            embl_nt = dbref.get('id') or embl_nt
            for prop in dbref.get('properties', []):
                if prop['key'].lower() in ['protein sequence id', 'proteinid', 'protein_id']:
                    embl_prot = prop['value']
    return refseq_prot, refseq_nt, embl_nt, embl_prot


def extract_contig_block(gb_text):
    """
    Extracts the full CONTIG block from a GenBank flatfile entry as a single string.
    """
    lines = gb_text.splitlines()
    contig_lines = []
    in_contig = False

    for line in lines:
        if line.lstrip().startswith('CONTIG'):
            in_contig = True
            contig_lines.append(line.strip())
        elif in_contig and (line.startswith(' ') or line.startswith('\t')):
            # Continuation line (indented)
            contig_lines.append(line.strip())
        elif in_contig and not (line.startswith(' ') or line.startswith('\t')):
            # End of CONTIG block
            break
    return " ".join(contig_lines) if contig_lines else ""


def get_contigs_from_refseq(refseq_nt, embl_nt):
    """Given a RefSeq nt accession, extract all contig accessions from CONTIG field. Fallback to master accession if none."""
    try:
        with Entrez.efetch(db='nuccore', id=refseq_nt, rettype='gb', retmode='text', timeout=60) as handle:
            gb_data = handle.read()
        
        contig_block = extract_contig_block(gb_data)
        if not contig_block:
            return [refseq_nt]  # fallback single

        # search for accession ids in join(xxx), e.g JFBT01000001.1
        contigs = re.findall(r'([A-Z]{4}\d{8}\.\d+)', contig_block)
        return contigs if contigs else [embl_nt]
    except Exception as e:
        print(f"Error in get_contigs_from_refseq({refseq_nt}): {e}")
        return [embl_nt]
    
    
def get_cds_list(embl_acc):
    """Get all CDS/gene features for a nucleotide accession."""
    genes = []
    try:
        with Entrez.efetch(db='nuccore', id=embl_acc, rettype='gbwithparts', retmode='text', timeout=500) as handle:
            records = list(SeqIO.parse(handle, "genbank"))
        for record in records:
            for idx, feat in enumerate(record.features):
                if feat.type == "CDS":
                    gene = feat.qualifiers.get('gene', [None])[0]
                    locus_tag = feat.qualifiers.get('locus_tag', [None])[0]
                    product = feat.qualifiers.get('product', [None])[0]
                    protein_id = feat.qualifiers.get('protein_id', [None])[0]
                    # db_xref
                    db_xrefs = feat.qualifiers.get('db_xref', [])
                    embl_prot = None
                    for dbx in db_xrefs:
                        if dbx.startswith("EMBL:"):
                            embl_prot = dbx.split("EMBL:")[1]
                        if dbx.startswith("protein_id:"):
                            embl_prot = dbx.split("protein_id:")[1]
                    start = int(feat.location.start)
                    end = int(feat.location.end)
                    strand = feat.location.strand
                    genes.append(dict(
                        gene=gene,
                        locus_tag=locus_tag,
                        product=product,
                        embl_prot=embl_prot,
                        protein_id=protein_id,
                        idx=idx,
                        start=start,
                        end=end,
                        mid=(start+end)//2,
                        contig=record.id,
                        strand=strand
                    ))
        return genes
    except Exception as e:
        print(f"Error parsing {embl_acc}: {e}")
        return []

# Main pipeline

In [ ]:
df = pd.read_csv(input_csv, header=None, names=['uniprot'])
results = []

for uniprot_id in tqdm(df['uniprot'], desc="UniProt IDs"):
    refseq_prot, refseq_nt, embl_nt, embl_prot = get_uniprot_mappings(uniprot_id)
    sleep(0.2)

    nucleotide_id = refseq_nt or embl_nt
    if not nucleotide_id:
        results.append({
            'uniprot': uniprot_id,
            'refseq_prot': refseq_prot or "",
            'refseq_nt': refseq_nt or "",
            'embl_nucleotide': embl_nt or "",
            'embl_protein': embl_prot or "",
            'error': 'No nucleotide sequences found'
        })
        continue

    contigs = get_contigs_from_refseq(nucleotide_id, embl_nt)
    sleep(0.2)

    # For each contig, scan for genes and annotate which contig they're from
    all_genes = []
    for ctg in contigs:
        genes = get_cds_list(ctg)
        for g in genes:
            g['contig'] = ctg
        all_genes.extend(genes)
        sleep(0.3)

    # Find the hit gene by protein_id match (embl_prot), if possible
    hit = next((g for g in all_genes if g['protein_id'] == embl_prot and embl_prot), None)
    
    row = dict(
        uniprot=uniprot_id,
        refseq_prot=refseq_prot or "",
        refseq_nt=refseq_nt or "",
        embl_nucleotide=embl_nt or "",
        embl_protein=embl_prot or "",
        total_contigs=len(contigs),
        contig_list=','.join(contigs),
    )

    if hit:
        row.update({
            "hit_gene": f"{hit['gene']}|{hit['locus_tag']}|{hit['start']}-{hit['end']}",
            "hit_contig": hit['contig'],
            "hit_start": hit['start'],
            "hit_end": hit['end'],
            "hit_mid": hit['mid'],
            "hit_strand": hit['strand'],
            "hit_protein_id": hit['protein_id']
        })
        hit_idx, hit_mid, hit_contig = hit['idx'], hit['mid'], hit['contig']
    else:
        row["hit_gene"] = ""
        row["hit_contig"] = ""
        row["hit_start"] = ""
        row["hit_end"] = ""
        row["hit_mid"] = ""
        row["hit_strand"] = ""
        row["hit_protein_id"] = ""
        hit_idx, hit_mid, hit_contig = None, None, None

    # Check: are ANY CDS features annotated with a gene name?
    no_gene_names_provided = all(g['gene'] is None for g in all_genes) if all_genes else True
    
    # For each search gene, find closest (prefer same contig), report distances if on same contig
    for sj in search_genes:
        if no_gene_names_provided:
            # No annotation present at all
            row[f"{sj}_gene"] = "no gene names provided"
            row[f"{sj}_contig"] = ""
            row[f"{sj}_dist_bp"] = ""
            row[f"{sj}_dist_genes"] = ""
            row[f"{sj}_start"] = ""
            row[f"{sj}_end"] = ""
            row[f"{sj}_strand"] = ""
            row[f"{sj}_protein_id"] = ""
        else:
            # Annotated genes present, see if our search gene exists
            sj_matches = [g for g in all_genes if g['gene'] and sj.lower() == g['gene'].lower()]
            if not sj_matches:
                row[f"{sj}_gene"] = "gene not in sequence"
                row[f"{sj}_contig"] = ""
                row[f"{sj}_dist_bp"] = ""
                row[f"{sj}_dist_genes"] = ""
                row[f"{sj}_start"] = ""
                row[f"{sj}_end"] = ""
                row[f"{sj}_strand"] = ""
                row[f"{sj}_protein_id"] = ""
            else:
                if hit is None:
                    sj_best = sj_matches[0]
                else:
                    sj_best = min(
                        sj_matches,
                        key=lambda g: (g['contig'] != hit_contig, abs(g['mid']-hit_mid))
                    )
                row[f"{sj}_gene"] = f"{sj_best['gene']}|{sj_best['locus_tag']}|{sj_best['start']}-{sj_best['end']}"
                row[f"{sj}_contig"] = sj_best['contig']
                row[f"{sj}_start"] = sj_best['start']
                row[f"{sj}_end"] = sj_best['end']
                row[f"{sj}_strand"] = sj_best['strand']
                row[f"{sj}_protein_id"] = sj_best['protein_id']
                if hit is None or sj_best['contig'] != hit_contig:
                    row[f"{sj}_dist_bp"] = "NA"
                    row[f"{sj}_dist_genes"] = "NA"
                else:
                    row[f"{sj}_dist_bp"] = abs(sj_best['mid']-hit_mid)
                    row[f"{sj}_dist_genes"] = sj_best['idx']-hit_idx

    results.append(row)

# Save results
pd.DataFrame(results).to_csv(output_csv, index=False)
print(f"Done! Results written to {output_csv}")

# Helper function (safe requests)

In [2]:
def safe_requests_get(url, timeout=10, tries=3, backoff=2):
    delay = 10
    for i in range(tries):
        try:
            r = requests.get(url, timeout=timeout)
            r.raise_for_status()
            return r
        except (requests.RequestException, Exception) as e:
            if i >= tries-1:
                print(f"ERROR: {e} for {url}")
                return None
            print(f"Retrying ({i+1}): {e} for {url}, sleeping {delay}s...")
            sleep(delay)
            delay *= backoff

def safe_entrez_efetch(*args, tries=3, backoff=2, **kwargs):
    delay = 10
    for i in range(tries):
        try:
            return Entrez.efetch(*args, **kwargs)
        except Exception as e:
            if i >= tries-1:
                print(f"ERROR: {e} for efetch with args {args} {kwargs}")
                return None
            print(f"Retrying efetch ({i+1}) for {args}: {e}, sleeping {delay}s...")
            sleep(delay)
            delay *= backoff


def get_uniprot_mappings(uniprot_id):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    r = safe_requests_get(url)
    if r is None:
        print(f"UniProt fetch failed for {uniprot_id}")
        return None, None, None, None
    data = r.json()
    refseq_prot, refseq_nt, embl_nt, embl_prot = None, None, None, None
    for dbref in data.get('uniProtKBCrossReferences', []):
        if dbref['database'] == "RefSeq":
            refseq_prot = dbref.get('id') or refseq_prot
            for prop in dbref.get('properties', []):
                if prop['key'] == 'NucleotideSequenceId':
                    refseq_nt = prop['value']
        if dbref['database'] == "EMBL":
            embl_nt = dbref.get('id') or embl_nt
            for prop in dbref.get('properties', []):
                if prop['key'].lower() in ['protein sequence id', 'proteinid', 'protein_id']:
                    embl_prot = prop['value']
    return refseq_prot, refseq_nt, embl_nt, embl_prot


def extract_contig_block(gb_text):
    lines = gb_text.splitlines()
    contig_lines = []
    in_contig = False
    for line in lines:
        if line.lstrip().startswith('CONTIG'):
            in_contig = True
            contig_lines.append(line.strip())
        elif in_contig and (line.startswith(' ') or line.startswith('\t')):
            contig_lines.append(line.strip())
        elif in_contig and not (line.startswith(' ') or line.startswith('\t')):
            break
    return " ".join(contig_lines) if contig_lines else ""


def get_contigs_from_refseq(refseq_nt, embl_nt):
    try:
        handle = safe_entrez_efetch(db='nuccore', id=refseq_nt, rettype='gb', retmode='text', timeout=60)
        if handle is None:
            return [embl_nt]
        gb_data = handle.read()
        handle.close()
        contig_block = extract_contig_block(gb_data)
        if not contig_block:
            return [refseq_nt]
        contigs = re.findall(r'([A-Z]{4}\d{8}\.\d+)', contig_block)
        return contigs if contigs else [embl_nt]
    except Exception as e:
        print(f"Error in get_contigs_from_refseq({refseq_nt}): {e}")
        return [embl_nt]


def get_cds_list(embl_acc, tries=3):
    genes = []
    delay = 1
    for attempt in range(tries):
        try:
            handle = safe_entrez_efetch(db='nuccore', id=embl_acc, rettype='gbwithparts', retmode='text', timeout=500)
            if handle is None:
                return []
            records = list(SeqIO.parse(handle, "genbank"))
            handle.close()
            for record in records:
                for idx, feat in enumerate(record.features):
                    if feat.type == "CDS":
                        gene = feat.qualifiers.get('gene', [None])[0]
                        locus_tag = feat.qualifiers.get('locus_tag', [None])[0]
                        product = feat.qualifiers.get('product', [None])[0]
                        protein_id = feat.qualifiers.get('protein_id', [None])[0]
                        db_xrefs = feat.qualifiers.get('db_xref', [])
                        embl_prot = None
                        for dbx in db_xrefs:
                            if dbx.startswith("EMBL:"):
                                embl_prot = dbx.split("EMBL:")[1]
                            if dbx.startswith("protein_id:"):
                                embl_prot = dbx.split("protein_id:")[1]
                        start = int(feat.location.start)
                        end = int(feat.location.end)
                        strand = feat.location.strand
                        genes.append(dict(
                            gene=gene,
                            locus_tag=locus_tag,
                            product=product,
                            embl_prot=embl_prot,
                            protein_id=protein_id,
                            idx=idx,
                            start=start,
                            end=end,
                            mid=(start+end)//2,
                            contig=record.id,
                            strand=strand
                        ))
            return genes
        except Exception as e:
            if attempt >= tries-1:
                print(f"Error parsing {embl_acc}: {e}")
                return []
            print(f"Retrying get_cds_list for {embl_acc}: {e}, sleeping {delay}s...")
            sleep(delay)
            delay *= 2
            
            
def save_checkpoint(df, results, output_csv):
    df_out = pd.DataFrame(results)
    df_out.to_csv(output_csv, index=False)
    print(f"\n[Checkpoint] Saved {len(results)} results to {output_csv}")
    

# Main function (safe requests)

In [3]:
df = pd.read_csv(input_csv, header=None, names=['uniprot'])
results = []

for i, uniprot_id in enumerate(tqdm(df['uniprot'], desc="UniProt IDs")):
    refseq_prot, refseq_nt, embl_nt, embl_prot = get_uniprot_mappings(uniprot_id)
    sleep(0.3)

    nucleotide_id = refseq_nt or embl_nt
    if not nucleotide_id:
        results.append({
            'uniprot': uniprot_id,
            'refseq_prot': refseq_prot or "",
            'refseq_nt': refseq_nt or "",
            'embl_nucleotide': embl_nt or "",
            'embl_protein': embl_prot or "",
            'error': 'No nucleotide sequences found'
        })
        continue

    contigs = get_contigs_from_refseq(nucleotide_id, embl_nt)
    sleep(0.3)

    all_genes = []
    for ctg in contigs:
        genes = get_cds_list(ctg)
        for g in genes:
            g['contig'] = ctg
        all_genes.extend(genes)
        sleep(0.4)

    hit = next((g for g in all_genes if g['protein_id'] == embl_prot and embl_prot), None)

    row = dict(
        uniprot=uniprot_id,
        refseq_prot=refseq_prot or "",
        refseq_nt=refseq_nt or "",
        embl_nucleotide=embl_nt or "",
        embl_protein=embl_prot or "",
        total_contigs=len(contigs),
        contig_list=','.join(contigs),
    )

    if hit:
        row.update({
            "hit_gene": f"{hit['gene']}|{hit['locus_tag']}|{hit['start']}-{hit['end']}",
            "hit_contig": hit['contig'],
            "hit_start": hit['start'],
            "hit_end": hit['end'],
            "hit_mid": hit['mid'],
            "hit_strand": hit['strand'],
            "hit_protein_id": hit['protein_id']
        })
        hit_idx, hit_mid, hit_contig = hit['idx'], hit['mid'], hit['contig']
    else:
        row["hit_gene"] = ""
        row["hit_contig"] = ""
        row["hit_start"] = ""
        row["hit_end"] = ""
        row["hit_mid"] = ""
        row["hit_strand"] = ""
        row["hit_protein_id"] = ""
        hit_idx, hit_mid, hit_contig = None, None, None

    no_gene_names_provided = all(g['gene'] is None for g in all_genes) if all_genes else True

    for sj in search_genes:
        if no_gene_names_provided:
            row[f"{sj}_gene"] = "no gene names provided"
            row[f"{sj}_contig"] = ""
            row[f"{sj}_dist_bp"] = ""
            row[f"{sj}_dist_genes"] = ""
            row[f"{sj}_start"] = ""
            row[f"{sj}_end"] = ""
            row[f"{sj}_strand"] = ""
            row[f"{sj}_protein_id"] = ""
        else:
            sj_matches = [g for g in all_genes if g['gene'] and sj.lower() == g['gene'].lower()]
            if not sj_matches:
                row[f"{sj}_gene"] = "gene not in sequence"
                row[f"{sj}_contig"] = ""
                row[f"{sj}_dist_bp"] = ""
                row[f"{sj}_dist_genes"] = ""
                row[f"{sj}_start"] = ""
                row[f"{sj}_end"] = ""
                row[f"{sj}_strand"] = ""
                row[f"{sj}_protein_id"] = ""
            else:
                if hit is None:
                    sj_best = sj_matches[0]
                else:
                    sj_best = min(
                        sj_matches,
                        key=lambda g: (g['contig'] != hit_contig, abs(g['mid']-hit_mid))
                    )
                row[f"{sj}_gene"] = f"{sj_best['gene']}|{sj_best['locus_tag']}|{sj_best['start']}-{sj_best['end']}"
                row[f"{sj}_contig"] = sj_best['contig']
                row[f"{sj}_start"] = sj_best['start']
                row[f"{sj}_end"] = sj_best['end']
                row[f"{sj}_strand"] = sj_best['strand']
                row[f"{sj}_protein_id"] = sj_best['protein_id']
                if hit is None or sj_best['contig'] != hit_contig:
                    row[f"{sj}_dist_bp"] = "NA"
                    row[f"{sj}_dist_genes"] = "NA"
                else:
                    row[f"{sj}_dist_bp"] = abs(sj_best['mid']-hit_mid)
                    row[f"{sj}_dist_genes"] = sj_best['idx']-hit_idx

    results.append(row)

    # --- Checkpoint save ---
    if (i + 1) % CHECKPOINT_EVERY == 0 or (i + 1) == len(df):
        save_checkpoint(df, results, output_csv)

print(f"Done! Results written to {output_csv}")

UniProt IDs:   2%|▏         | 20/987 [03:41<3:14:04, 12.04s/it]


[Checkpoint] Saved 20 results to protein_synteny_topmiddleClusterIDs_output.csv


UniProt IDs:   4%|▍         | 40/987 [05:50<1:27:50,  5.57s/it]


[Checkpoint] Saved 40 results to protein_synteny_topmiddleClusterIDs_output.csv


UniProt IDs:   6%|▌         | 60/987 [07:30<1:12:23,  4.69s/it]

...


[Checkpoint] Saved 960 results to protein_synteny_topmiddleClusterIDs_output.csv


UniProt IDs:  99%|█████████▉| 980/987 [3:30:47<00:54,  7.84s/it]     


[Checkpoint] Saved 980 results to protein_synteny_topmiddleClusterIDs_output.csv


UniProt IDs: 100%|██████████| 987/987 [3:31:04<00:00, 12.83s/it]


[Checkpoint] Saved 987 results to protein_synteny_topmiddleClusterIDs_output.csv
Done! Results written to protein_synteny_topmiddleClusterIDs_output.csv
